In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

import pandas as pd 

## Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

## Attempt 1

In [2]:
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
application_df = application_df.drop(['EIN','NAME'],axis=1)

app_counts = application_df['APPLICATION_TYPE'].value_counts()
replace_app = list(app_counts[app_counts < 500].index)
# Replace in dataframe
for app in replace_app:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")

class_counts = application_df['CLASSIFICATION'].value_counts()
replace_class = list(class_counts[class_counts < 1883].index)
# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


### Attempt 1: Binning additional columns and additonal modifications

In [4]:
aff_counts = application_df['AFFILIATION'].value_counts()
aff_counts

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [5]:
replace_aff = list(aff_counts[aff_counts < 100].index)
# Replace in dataframe
for aff in replace_aff:
    application_df.AFFILIATION = application_df.AFFILIATION.replace(aff,"Other")
    
# Check to make sure binning was successful
application_df.AFFILIATION.value_counts()

Independent         18480
CompanySponsored    15705
Other                 114
Name: AFFILIATION, dtype: int64

In [6]:
use_counts = application_df['USE_CASE'].value_counts()
use_counts

Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: USE_CASE, dtype: int64

In [7]:
replace_use = list(use_counts[use_counts < 500].index)
# Replace in dataframe
for use in replace_use:
    application_df.USE_CASE = application_df.USE_CASE.replace(use,"Other")
    
# Check to make sure binning was successful
application_df.USE_CASE.value_counts()

Preservation    28095
ProductDev       5671
Other             533
Name: USE_CASE, dtype: int64

In [8]:
org_counts = application_df['ORGANIZATION'].value_counts()
org_counts

Trust           23515
Association     10255
Co-operative      486
Corporation        43
Name: ORGANIZATION, dtype: int64

In [9]:
replace_org = list(org_counts[org_counts < 500].index)
# Replace in dataframe
for org in replace_org:
    application_df.ORGANIZATION = application_df.ORGANIZATION.replace(org,"Other")
    
# Check to make sure binning was successful
application_df.ORGANIZATION.value_counts()

Trust          23515
Association    10255
Other            529
Name: ORGANIZATION, dtype: int64

In [10]:
inc_counts = application_df['INCOME_AMT'].value_counts()
inc_counts

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [11]:
application_df['INCOME_AMT'].mask(application_df['INCOME_AMT'] == '1M-5M', '1M+', inplace=True)
application_df['INCOME_AMT'].mask(application_df['INCOME_AMT'] == '5M-10M', '1M+', inplace=True)
application_df['INCOME_AMT'].mask(application_df['INCOME_AMT'] == '10M-50M', '1M+', inplace=True)
application_df['INCOME_AMT'].mask(application_df['INCOME_AMT'] == '50M+', '1M+', inplace=True)
inc_counts = application_df['INCOME_AMT'].value_counts()
inc_counts

0                24388
25000-99999       3747
100000-499999     3374
1M+               1519
1-9999             728
10000-24999        543
Name: INCOME_AMT, dtype: int64

In [12]:
spec_counts = application_df['SPECIAL_CONSIDERATIONS'].value_counts()
spec_counts

N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [13]:
application_df = application_df.drop(['SPECIAL_CONSIDERATIONS'],axis=1)

In [14]:
cat_var = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
cat_var

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT']

In [15]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
#  YOUR CODE GOES HERE
encode_df = pd.DataFrame(enc.fit_transform(application_df[cat_var]))
# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(cat_var)
encode_df.head()

C:\Users\tmril\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Other,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M+,INCOME_AMT_25000-99999
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [16]:
# Merge one-hot encoded features and drop the originals
#  YOUR CODE GOES HERE
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(cat_var,1)
application_df.head()

C:\Users\tmril\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Other,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M+,INCOME_AMT_25000-99999
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [17]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

C:\Users\tmril\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


In [18]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [19]:
# Define the model
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5
nnc1 = tf.keras.models.Sequential()

# First hidden layer
nnc1.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)
# Second hidden layer
nnc1.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nnc1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nnc1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 264       
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 315
Trainable params: 315
Non-trainable params: 0
_________________________________________________________________


In [20]:
## Define the checkpoint path and filenames
os.makedirs("checkpoints/optimization1/",exist_ok=True)
checkpoint_path = "checkpoints/optimization1/weights.{epoch:02d}.hdf5"
## Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=5,    
)
# Compile the model
nnc1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [21]:
# Train the model
fit_model = nnc1.fit(X_train_scaled, y_train, epochs=100,callbacks=[cp_callback]) 

Epoch 1/100
804/804 [==============================] - 1s 923us/step - loss: 0.6183 - accuracy: 0.6797
Epoch 2/100
804/804 [==============================] - 1s 911us/step - loss: 0.5624 - accuracy: 0.7267
Epoch 3/100
804/804 [==============================] - 1s 907us/step - loss: 0.5573 - accuracy: 0.7289
Epoch 4/100
804/804 [==============================] - 1s 906us/step - loss: 0.5552 - accuracy: 0.7304
Epoch 5/100
774/804 [===========================>..] - ETA: 0s - loss: 0.5534 - accuracy: 0.7309
Epoch 5: saving model to checkpoints/optimization1\weights.05.hdf5
804/804 [==============================] - 1s 932us/step - loss: 0.5536 - accuracy: 0.7304
Epoch 6/100
804/804 [==============================] - 1s 911us/step - loss: 0.5522 - accuracy: 0.7304
Epoch 7/100
804/804 [==============================] - 1s 939us/step - loss: 0.5519 - accuracy: 0.7303
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5512 - accuracy: 0.7312
Epoch 9/100
804/804 [=====

804/804 [==============================] - 1s 1ms/step - loss: 0.5460 - accuracy: 0.7349
Epoch 63/100
804/804 [==============================] - 1s 966us/step - loss: 0.5462 - accuracy: 0.7336
Epoch 64/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5457 - accuracy: 0.7344
Epoch 65/100
782/804 [============================>.] - ETA: 0s - loss: 0.5460 - accuracy: 0.7350
Epoch 65: saving model to checkpoints/optimization1\weights.65.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5460 - accuracy: 0.7351
Epoch 66/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5460 - accuracy: 0.7351
Epoch 67/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5459 - accuracy: 0.7353
Epoch 68/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5459 - accuracy: 0.7350
Epoch 69/100
804/804 [==============================] - 1s 986us/step - loss: 0.5458 - accuracy: 0.7352
Epoch 70/100
773/804 [==================

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nnc1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5525 - accuracy: 0.7269 - 354ms/epoch - 1ms/step
Loss: 0.552536129951477, Accuracy: 0.7268804907798767


## 2nd Attempt

In [23]:
application_df2 = pd.read_csv("Resources/charity_data.csv")
application_df2.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [24]:
application_df2 = application_df2.drop(['EIN','NAME'],axis=1)
app_counts2 = application_df2['APPLICATION_TYPE'].value_counts()
replace_app2 = list(app_counts2[app_counts2 < 500].index)
# Replace in dataframe
for app in replace_app2:
    application_df2.APPLICATION_TYPE = application_df2.APPLICATION_TYPE.replace(app,"Other")

class_counts2 = application_df2['CLASSIFICATION'].value_counts()
replace_class2 = list(class_counts2[class_counts2 < 1883].index)
# Replace in dataframe
for cls in replace_class2:
    application_df2.CLASSIFICATION = application_df2.CLASSIFICATION.replace(cls,"Other")
application_df2.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [25]:
cat_var2 = application_df2.dtypes[application_df2.dtypes == "object"].index.tolist()
cat_var2

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [26]:
# Create a OneHotEncoder instance
enc2 = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df2 = pd.DataFrame(enc2.fit_transform(application_df2[cat_var2]))
# Add the encoded variable names to the dataframe
encode_df2.columns = enc2.get_feature_names(cat_var2)
encode_df2.head()

C:\Users\tmril\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [27]:
application_df2 = application_df2.merge(encode_df2,left_index=True, right_index=True)
application_df2 = application_df2.drop(cat_var2,1)
application_df2.head()

C:\Users\tmril\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [28]:
# Split our preprocessed data into our features and target arrays
y2 = application_df2["IS_SUCCESSFUL"].values
X2 = application_df2.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, random_state=78)

C:\Users\tmril\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [29]:
# Create a StandardScaler instances
scaler2 = StandardScaler()

# Fit the StandardScaler
X_scaler2 = scaler2.fit(X_train2)

# Scale the data
X_train_scaled2 = X_scaler2.transform(X_train2)
X_test_scaled2 = X_scaler2.transform(X_test2)

### Adding Neurons to hidden layers and adding another hidden layer

In [30]:
# Define the model - deep neural net
number_input_features2 = len(X_train2[0])
hidden_nodes_layer1_2 = 80
hidden_nodes_layer2_2 = 50
hidden_nodes_layer3_2 = 10
nnc2 = tf.keras.models.Sequential()
# First hidden layer
nnc2.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1_2, input_dim=number_input_features2, activation="relu")
)
# Second hidden layer
nnc2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2_2, activation="relu"))
# Third hidden layer
nnc2.add(tf.keras.layers.Dense(units=hidden_nodes_layer3_2, activation="relu"))

# Output layer
nnc2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nnc2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                3520      
                                                                 
 dense_4 (Dense)             (None, 50)                4050      
                                                                 
 dense_5 (Dense)             (None, 10)                510       
                                                                 
 dense_6 (Dense)             (None, 1)                 11        
                                                                 
Total params: 8,091
Trainable params: 8,091
Non-trainable params: 0
_________________________________________________________________


In [31]:
## Define the checkpoint path and filenames
os.makedirs("checkpoints/optimization2/",exist_ok=True)
checkpoint_path2 = "checkpoints/optimization2/weights.{epoch:02d}.hdf5"
## Create a callback that saves the model's weights every 5 epochs
cp_callback2 = ModelCheckpoint(
    filepath=checkpoint_path2,
    verbose=1,
    save_weights_only=True,
    period=5,    
)
# Compile the model
nnc2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [32]:
# Train the model
fit_model2 = nnc2.fit(X_train_scaled2, y_train2, epochs=100,callbacks=[cp_callback2]) 

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5691 - accuracy: 0.7233
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5535 - accuracy: 0.7307
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5507 - accuracy: 0.7321
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5498 - accuracy: 0.7329
Epoch 5/100
804/804 [==============================] - ETA: 0s - loss: 0.5486 - accuracy: 0.7337
Epoch 5: saving model to checkpoints/optimization2\weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5486 - accuracy: 0.7337
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5477 - accuracy: 0.7336
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5468 - accuracy: 0.7342
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5467 - accuracy: 0.7332
Epoch 9/100
804/804 [===================

804/804 [==============================] - 1s 1ms/step - loss: 0.5371 - accuracy: 0.7400
Epoch 64/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5367 - accuracy: 0.7402
Epoch 65/100
785/804 [============================>.] - ETA: 0s - loss: 0.5366 - accuracy: 0.7401
Epoch 65: saving model to checkpoints/optimization2\weights.65.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5366 - accuracy: 0.7397
Epoch 66/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5360 - accuracy: 0.7402
Epoch 67/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5366 - accuracy: 0.7391
Epoch 68/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5362 - accuracy: 0.7410
Epoch 69/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5359 - accuracy: 0.7409
Epoch 70/100
776/804 [===========================>..] - ETA: 0s - loss: 0.5366 - accuracy: 0.7400
Epoch 70: saving model to checkpoints/optimizati

In [33]:
# Evaluate the model using the test data
model_loss2, model_accuracy2 = nnc2.evaluate(X_test_scaled2,y_test2,verbose=2)
print(f"Loss: {model_loss2}, Accuracy: {model_accuracy2}")

268/268 - 0s - loss: 0.5590 - accuracy: 0.7238 - 367ms/epoch - 1ms/step
Loss: 0.5589742660522461, Accuracy: 0.7238484025001526


## Attempt 3

### Changing Activation Functions in each hidden layer

In [34]:
# Define the model - deep neural net
number_input_features = len(X_train2[0])
hidden_nodes_layer1_3 = 80
hidden_nodes_layer2_3 = 50
hidden_nodes_layer3_3 = 10
nnc3 = tf.keras.models.Sequential()
# First hidden layer
nnc3.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1_3, input_dim=number_input_features, activation="leaky_relu")
)
# Second hidden layer
nnc3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2_3, activation="elu"))
# Third hidden layer
nnc3.add(tf.keras.layers.Dense(units=hidden_nodes_layer3_3, activation="selu"))

# Output layer
nnc3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nnc3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 80)                3520      
                                                                 
 dense_8 (Dense)             (None, 50)                4050      
                                                                 
 dense_9 (Dense)             (None, 10)                510       
                                                                 
 dense_10 (Dense)            (None, 1)                 11        
                                                                 
Total params: 8,091
Trainable params: 8,091
Non-trainable params: 0
_________________________________________________________________


In [35]:
## Define the checkpoint path and filenames
os.makedirs("checkpoints/optimization3/",exist_ok=True)
checkpoint_path3 = "checkpoints/optimization3/weights.{epoch:02d}.hdf5"

## Create a callback that saves the model's weights every 5 epochs
cp_callback3 = ModelCheckpoint(
    filepath=checkpoint_path3,
    verbose=1,
    save_weights_only=True,
    period=5,    
)

In [36]:
nnc3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

### Increase number of epochs

In [37]:
fit_model3 = nnc3.fit(X_train_scaled2, y_train2, epochs=200,callbacks=[cp_callback3]) 

Epoch 1/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5734 - accuracy: 0.7193
Epoch 2/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5569 - accuracy: 0.7293
Epoch 3/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5541 - accuracy: 0.7300
Epoch 4/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5519 - accuracy: 0.7311
Epoch 5/200
792/804 [============================>.] - ETA: 0s - loss: 0.5512 - accuracy: 0.7325
Epoch 5: saving model to checkpoints/optimization3\weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5516 - accuracy: 0.7317
Epoch 6/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5506 - accuracy: 0.7310
Epoch 7/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5495 - accuracy: 0.7316
Epoch 8/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5491 - accuracy: 0.7331
Epoch 9/200
804/804 [===================

804/804 [==============================] - 1s 1ms/step - loss: 0.5392 - accuracy: 0.7388
Epoch 64/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5387 - accuracy: 0.7388
Epoch 65/200
794/804 [============================>.] - ETA: 0s - loss: 0.5385 - accuracy: 0.7387
Epoch 65: saving model to checkpoints/optimization3\weights.65.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5386 - accuracy: 0.7386
Epoch 66/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5387 - accuracy: 0.7388
Epoch 67/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5389 - accuracy: 0.7391
Epoch 68/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5388 - accuracy: 0.7389
Epoch 69/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5387 - accuracy: 0.7381
Epoch 70/200
778/804 [============================>.] - ETA: 0s - loss: 0.5391 - accuracy: 0.7386
Epoch 70: saving model to checkpoints/optimizati

765/804 [===========================>..] - ETA: 0s - loss: 0.5360 - accuracy: 0.7401
Epoch 125: saving model to checkpoints/optimization3\weights.125.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5361 - accuracy: 0.7399
Epoch 126/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5364 - accuracy: 0.7410
Epoch 127/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5362 - accuracy: 0.7396
Epoch 128/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5360 - accuracy: 0.7410
Epoch 129/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5361 - accuracy: 0.7408
Epoch 130/200
769/804 [===========================>..] - ETA: 0s - loss: 0.5362 - accuracy: 0.7406
Epoch 130: saving model to checkpoints/optimization3\weights.130.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5360 - accuracy: 0.7406
Epoch 131/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5364 - 

804/804 [==============================] - 1s 1ms/step - loss: 0.5352 - accuracy: 0.7407
Epoch 186/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5346 - accuracy: 0.7405
Epoch 187/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5347 - accuracy: 0.7405
Epoch 188/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5347 - accuracy: 0.7409
Epoch 189/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5345 - accuracy: 0.7409
Epoch 190/200
794/804 [============================>.] - ETA: 0s - loss: 0.5349 - accuracy: 0.7416
Epoch 190: saving model to checkpoints/optimization3\weights.190.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5344 - accuracy: 0.7418
Epoch 191/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5351 - accuracy: 0.7413
Epoch 192/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5346 - accuracy: 0.7410
Epoch 193/200
804/804 [============

In [38]:
# Evaluate the model using the test data
model_loss3, model_accuracy3 = nnc3.evaluate(X_test_scaled2,y_test2,verbose=2)
print(f"Loss: {model_loss3}, Accuracy: {model_accuracy3}")

268/268 - 0s - loss: 0.5564 - accuracy: 0.7270 - 345ms/epoch - 1ms/step
Loss: 0.5563811659812927, Accuracy: 0.7269970774650574


In [39]:
# Export our model to HDF5 file
nnc3.save("Resources/AlphabetSoupCharity_Optimization.h5")